# 20_Dependency_Injection

## 1. UsersModule – Wiring Controller & Service

```ts
import { Module } from "@nestjs/common";
import { UsersController } from "./users.controller";
import { UsersService } from './users.service';

@Module({
    controllers: [UsersController],
    providers: [UsersService]
})
export class UsersModule {
    
}
```

### Key Points

* **`@Module()` decorator** defines a logical block of the NestJS application.
* `controllers: [UsersController]`

  * Registers `UsersController` inside this module.
  * All routes defined in `UsersController` belong to this module.
* `providers: [UsersService]`

  * Registers `UsersService` as a **provider**.
  * Makes `UsersService` available for **dependency injection** inside this module (e.g., in `UsersController`).

> In simple words: **UsersModule = UsersController + UsersService wired together**.

---

## 2. UsersService – The Business Logic Layer

```ts
import { Injectable } from "@nestjs/common";

@Injectable()
export class UsersService{
    users: {id: number, name: string, email: string, gender: string, isMarried: boolean}[] = [
        {id: 1, name: 'Alice', email: 'Alice@example.com', gender: 'female', isMarried: true},
        {id: 2, name: 'Bob', email: 'Bob@example.com', gender: 'male', isMarried: false},
        {id: 3, name: 'Charlie', email: 'Charlie@example.com', gender: 'male', isMarried: true}
    ];

    getAllUsers() {
        return this.users;
    }   

    getUserById(id: number) {
        return this.users.find(user => user.id === id);
    }

    createUser(user: {id: number, name: string, email: string, gender: string, isMarried: boolean}) {
        this.users.push(user);
    }
}
```

### Important Concepts

* **`@Injectable()` decorator**

  * Tells NestJS that this class is a **provider**.
  * NestJS can now **create and inject** an instance of `UsersService` wherever it is needed.

* **In-memory data**

  * `users` array simulates a **temporary database**.
  * Only for learning/demo – in real apps, this logic typically talks to a real DB.

* **Service methods**

  * `getAllUsers()` → returns all users.
  * `getUserById(id: number)` → finds a user by `id`.
  * `createUser(user: ...)` → pushes a new user into the array.

> Summary: **Service = place for business logic & data handling.** Controller should not contain complex logic; it should call service methods.

---

## 3. UsersController – Handling Routes & Using the Service

```ts
import { Controller, Get, Param, ParseIntPipe, Post, Query, DefaultValuePipe, ValidationPipe, Body, Patch } from '@nestjs/common';
import { UsersService } from './users.service';
import { CreateUserDto } from './dtos/create-user.dto';
import { UpdateUserDto } from './dtos/update-user.dto';

@Controller('users')
export class UsersController {
    constructor(private usersService: UsersService){}

    @Get()
    getUsers(
        @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
        @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number
    ){
        console.log(limit, page)
        return this.usersService.getAllUsers();
    }

    @Get(':id')
    getUserById(@Param('id', ParseIntPipe) id: number){
        console.log(typeof id, id);
        return this.usersService.getUserById(id);    
    }

    @Post()
    createUser(@Body() user: CreateUserDto){
        this.usersService.createUser(user);
        return 'User created successfully';
    }

    @Patch()
    updateUser(@Body() body: UpdateUserDto){
        console.log(body);
    }
}
```

### 3.1 Controller Basics

* `@Controller('users')`

  * Base route for this controller = `/users`.
* **Constructor injection**

  ```ts
  constructor(private usersService: UsersService){}
  ```

  * Here, `UsersService` is injected automatically by NestJS.
  * `private usersService: UsersService`:

    * Creates a **private property** `usersService` in the class.
    * NestJS assigns an instance of `UsersService` to it.
  * This is exactly what **Dependency Injection** is.

---

## 4. Dependency Injection (DI) in NestJS – Concept & Steps

### 4.1 What is Dependency Injection?

* **Definition (simple):**

  * Instead of a class **creating its own dependencies** using `new`, those dependencies are **provided (injected)** from outside.
* In our case:

  * `UsersController` depends on `UsersService`.
  * Instead of `new UsersService()`, NestJS **injects** an instance.

### 4.2 Why Dependency Injection?

* **Loose coupling** – controller is not tightly bound to a concrete implementation.
* **Modular** – easy to replace `UsersService` (e.g., with a mock service in tests).
* **Testable** – we can inject mock dependencies when testing.
* **Maintainable & reusable** – cleaner architecture.

### 4.3 Steps to Use DI in NestJS (as in this example)

1. **Mark the service as injectable**

   ```ts
   @Injectable()
   export class UsersService { ... }
   ```

   * Tells NestJS: this can be used as a **provider**.

2. **Register the service in a module**

   ```ts
   @Module({
     controllers: [UsersController],
     providers: [UsersService],
   })
   export class UsersModule {}
   ```

   * Makes NestJS aware of `UsersService` inside this module.

3. **Inject the service in the controller constructor**

   ```ts
   constructor(private usersService: UsersService) {}
   ```

   * NestJS automatically **creates and injects** an instance.
   * No manual `new UsersService()`.

> Final picture: **Module provides the service → Controller requests it via constructor → NestJS injects it automatically.**

---

## 5. Route Handlers & Pipes in UsersController

### 5.1 GET /users – Query Params with Default & Pipes

```ts
@Get()
getUsers(
  @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
  @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
){
  console.log(limit, page);
  return this.usersService.getAllUsers();
}
```

* **Route:** `GET /users`
* **Query parameters:**

  * `limit` → number of items per page (default 10).
  * `page` → which page (default 1).
* **Pipes used:**

  * `DefaultValuePipe(10)`

    * If `limit` is not provided, it becomes `10`.
  * `ParseIntPipe`

    * Converts the query string value into a number.
* Example requests:

  * `/users` → `limit = 10`, `page = 1`.
  * `/users?limit=5&page=2` → `limit = 5`, `page = 2`.

> Currently the method ignores pagination and returns all users, but this is the typical pattern for **paginated APIs**.

---

### 5.2 GET /users/:id – Route Param with ParseIntPipe

```ts
@Get(':id')
getUserById(@Param('id', ParseIntPipe) id: number){
  console.log(typeof id, id);
  return this.usersService.getUserById(id);
}
```

* **Route:** `GET /users/:id`

  * Example: `/users/2`
* **`@Param('id', ParseIntPipe)`**

  * Extracts the `id` from the URL.
  * `ParseIntPipe` converts the string ID into a number.
* Uses `this.usersService.getUserById(id)` to fetch the user.

> Note: Without `ParseIntPipe`, `id` would be a string.

---

### 5.3 POST /users – Creating a User with DTO

```ts
@Post()
createUser(@Body() user: CreateUserDto){
  this.usersService.createUser(user);
  return 'User created successfully';
}
```

* **Route:** `POST /users`
* **`@Body()`**

  * Takes the JSON body from the request.
  * Typed as `CreateUserDto`.
* `CreateUserDto`

  * A DTO (Data Transfer Object) used to define and validate the shape of the incoming data.
  * Actual validations will work when you use `ValidationPipe` globally or per-route.

Example request body:

```json
{
  "id": 4,
  "name": "David",
  "email": "david@example.com",
  "gender": "male",
  "isMarried": false
}
```

---

### 5.4 PATCH /users – Updating User (Skeleton)

```ts
@Patch()
updateUser(@Body() body: UpdateUserDto){
  console.log(body);
}
```

* **Route:** `PATCH /users`
* Uses `UpdateUserDto` for incoming data.
* Currently only logs the body – in a real app you would:

  * Validate incoming data.
  * Call a service method like `this.usersService.updateUser(...)`.

---

## 6. Big Picture Recap (for this part of your notes)

1. **UsersModule**

   * Groups `UsersController` + `UsersService` together.
   * Declares `UsersService` in `providers` so it can be injected.

2. **UsersService**

   * Marked as `@Injectable()`.
   * Contains business logic and in-memory data.

3. **UsersController**

   * Uses `@Controller('users')` to define base route.
   * Injects `UsersService` using **Dependency Injection** via constructor.
   * Defines routes:

     * `GET /users` – list users (with `limit` & `page` query params + pipes).
     * `GET /users/:id` – get single user (with `ParseIntPipe`).
     * `POST /users` – create a new user using `CreateUserDto`.
     * `PATCH /users` – update user using `UpdateUserDto` (currently just logs).

4. **Dependency Injection Flow**

   * `@Injectable()` on service → service becomes a provider.
   * Service is registered in module `providers`.
   * Controller asks for the service in its constructor.
   * NestJS automatically injects an instance (no `new` keyword).

> This whole setup is a classic example of how NestJS encourages **clean architecture** using **modules, controllers, services, DTOs, and dependency injection**.
